# Notebook 02: AgentCore Runtime Setup

## Learning Objectives
- Configure AgentCore Runtime for travel agent
- Create basic conversational agent using Strands
- Implement multi-turn dialogue patterns
- Test conversation flow and state management

## Prerequisites
- Completed Notebook 01 (Foundation)
- AWS environment validated
- API keys configured

## Step 1: Connect to your AWS environment

In [ ]:
import os

os.environ['AWS_REGION'] = 'us-east-1'

# APPROACH A: Use credentials
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'
# os.environ['AWS_SESSION_TOKEN'] = "your_session_token"

# APPROACH B: Use AWS SSO profile
#os.environ['AWS_PROFILE'] = 'your_profile'
# Remove any existing credential env vars to force profile usage
#for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
#    os.environ.pop(key, None)

os.environ['AWS_REGION'] = 'us-east-1'

print("✅ AWS Profile set. Please restart kernel and run all cells.")

In [ ]:
import os
from dotenv import load_dotenv
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel

# Load environment variables
load_dotenv()

print("✅ AgentCore Runtime imports successful")

## Step 2: Create Basic Travel Agent Tools

In [ ]:
# Define basic travel tools for our agent
@tool
def get_travel_preferences():
    """Get user's travel preferences from memory"""
    # Mock implementation - will be enhanced in Memory notebook
    return {
        "hotel_type": "mid-range",
        "food_preference": "vegetarian",
        "budget_range": "moderate"
    }

@tool
def calculate_budget(total_budget: int, days: int):
    """Calculate daily budget allocation for travel"""
    # Basic budget breakdown
    daily_budget = total_budget / days
    allocation = {
        "flights": total_budget * 0.24,  # 24%
        "hotels": total_budget * 0.36,   # 36%
        "food": total_budget * 0.20,     # 20%
        "activities": total_budget * 0.16, # 16%
        "buffer": total_budget * 0.04     # 4%
    }
    return {
        "daily_budget": daily_budget,
        "allocation": allocation
    }

@tool
def get_destination_info(destination: str):
    """Get basic information about a travel destination"""
    # Mock implementation - will be enhanced with real APIs
    destinations = {
        "rome": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Colosseum", "Vatican", "Trevi Fountain"]
        },
        "florence": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Uffizi Gallery", "Ponte Vecchio", "Duomo"]
        },
        "venice": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["St. Mark's Square", "Grand Canal", "Doge's Palace"]
        }
    }
    return destinations.get(destination.lower(), {"error": "Destination not found"})

print("✅ Travel agent tools defined")

## Step 3: Create Travel Agent with Bedrock Model

In [ ]:
# Initialize Bedrock model for the agent
model_id = "us.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(model_id=model_id)

# Create the travel agent with system prompt
system_prompt = """
You are an AI Travel Companion specializing in planning trips to Italy. 
Your expertise includes:
- Flight and hotel recommendations
- Budget optimization and allocation
- Destination information and attractions
- Personalized recommendations based on user preferences

Always ask clarifying questions to better understand the user's needs.
Be helpful, friendly, and provide detailed explanations for your recommendations.
Remember user preferences and reference them in future interactions.
"""

travel_agent = Agent(
    model=model,
    tools=[get_travel_preferences, calculate_budget, get_destination_info],
    system_prompt=system_prompt
)

print("✅ Travel agent created with Bedrock model")

## Step 4: Test Local Agent Functionality

In [ ]:
# Test the agent locally before deploying to runtime
def test_travel_agent(user_input):
    """Test function for local agent invocation"""
    print(f"User: {user_input}")
    response = travel_agent(user_input)
    agent_response = response.message['content'][0]['text']
    print(f"Agent: {agent_response}")
    return agent_response

# Test basic functionality
print("🧪 Testing Travel Agent Locally")
print("=" * 50)

test_travel_agent("Hi, I want to plan a trip to Italy")

In [ ]:
# Test budget calculation
test_travel_agent("I have a budget of $5000 for a 10-day trip. How should I allocate it?")

In [ ]:
# Test destination information
test_travel_agent("Tell me about Rome and what I should see there")

## Step 5: Prepare Agent for AgentCore Runtime

In [ ]:
%%writefile ../backend/runtime/simple_agent/travel_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import json

# Initialize AgentCore Runtime App
app = BedrockAgentCoreApp()

# Define travel tools
@tool
def get_travel_preferences():
    """Get user's travel preferences from memory"""
    return {
        "hotel_type": "mid-range",
        "food_preference": "vegetarian",
        "budget_range": "moderate"
    }

@tool
def calculate_budget(total_budget: int, days: int):
    """Calculate daily budget allocation for travel"""
    daily_budget = total_budget / days
    allocation = {
        "flights": total_budget * 0.24,
        "hotels": total_budget * 0.36,
        "food": total_budget * 0.20,
        "activities": total_budget * 0.16,
        "buffer": total_budget * 0.04
    }
    return {
        "daily_budget": daily_budget,
        "allocation": allocation
    }

@tool
def get_destination_info(destination: str):
    """Get basic information about a travel destination"""
    destinations = {
        "rome": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Colosseum", "Vatican", "Trevi Fountain"]
        },
        "florence": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["Uffizi Gallery", "Ponte Vecchio", "Duomo"]
        },
        "venice": {
            "country": "Italy",
            "currency": "EUR",
            "language": "Italian",
            "attractions": ["St. Mark's Square", "Grand Canal", "Doge's Palace"]
        }
    }
    return destinations.get(destination.lower(), {"error": "Destination not found"})

# Initialize model and agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

system_prompt = """
You are an AI Travel Companion specializing in planning trips to Italy. 
Your expertise includes:
- Flight and hotel recommendations
- Budget optimization and allocation
- Destination information and attractions
- Personalized recommendations based on user preferences

Always ask clarifying questions to better understand the user's needs.
Be helpful, friendly, and provide detailed explanations for your recommendations.
Remember user preferences and reference them in future interactions.
"""

travel_agent = Agent(
    model=model,
    tools=[get_travel_preferences, calculate_budget, get_destination_info],
    system_prompt=system_prompt
)

@app.entrypoint
def invoke_travel_agent(payload):
    """AgentCore Runtime entrypoint for travel agent"""
    user_input = payload.get("prompt", "")
    print(f"User input: {user_input}")
    
    response = travel_agent(user_input)
    agent_response = response.message['content'][0]['text']
    
    return agent_response

if __name__ == "__main__":
    app.run()

In [ ]:
%%writefile ../backend/runtime/simple_agent/requirements.txt
# Core Amazon Bedrock AgentCore dependencies
bedrock-agentcore>=1.0.5
bedrock-agentcore-starter-toolkit>=0.1.27

# AWS SDK and utilities
boto3>=1.40.62
python-dotenv>=1.2.1

# Agent framework
strands-agents>=1.14.0

## Step 6: Deploy to AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os
from pathlib import Path

# Initialize runtime deployment
boto_session = Session()
region = boto_session.region_name or "us-east-1"

# Change to project root for AgentCore configuration
original_dir = os.getcwd()
os.chdir('../backend/runtime/simple_agent')
project_root = os.getcwd()

agentcore_runtime = Runtime()
agent_name = "travel_companion_basic"

print(f"🚀 Configuring AgentCore Runtime deployment...")
print(f"Agent Name: {agent_name}")
print(f"Region: {region}")
print(f"Project Root: {project_root}")

# Configure with paths relative to project root
configure_response = agentcore_runtime.configure(
    entrypoint="travel_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    container_runtime=None  # Force CodeBuild for cross-platform build
)

print("✅ Runtime configuration complete")
print("💡 Using CodeBuild for cross-platform ARM64 deployment")
configure_response

In [ ]:
# Launch the agent to AgentCore Runtime
print("🚀 Launching agent to AgentCore Runtime...")
print("This may take 5-10 minutes...")

launch_result = agentcore_runtime.launch()
print("✅ Launch initiated")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"ECR URI: {launch_result.ecr_uri}")

In [ ]:
# Check deployment status
import time

print("⏳ Checking deployment status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    print(f"Status: {status}")
    time.sleep(30)  # Check every 30 seconds
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

print(f"\n🎉 Final Status: {status}")

if status == 'READY':
    print("✅ Agent successfully deployed to AgentCore Runtime!")
else:
    print("❌ Deployment failed. Check AWS console for details.")

# Return to original directory
os.chdir(original_dir)

## Step 7: Test Deployed Agent

In [ ]:
# Test the deployed agent
if status == 'READY':
    print("🧪 Testing deployed Travel Agent")
    print("=" * 50)
    
    # Test basic interaction
    test_payload = {"prompt": "Hi, I want to plan a 10-day trip to Italy with a $5000 budget"}
    invoke_response = agentcore_runtime.invoke(test_payload)
    
    print(f"User: {test_payload['prompt']}")
    print(f"Agent: {invoke_response['response'][0]}")
else:
    print("⚠️ Cannot test - deployment not ready")

In [ ]:
# Test budget calculation functionality
if status == 'READY':
    test_payload = {"prompt": "Can you help me allocate my $5000 budget for 10 days?"}
    invoke_response = agentcore_runtime.invoke(test_payload)
    
    print(f"\nUser: {test_payload['prompt']}")
    print(f"Agent: {invoke_response['response'][0]}")

## Step 8: Multi-turn Conversation Test

In [ ]:
# Test multi-turn conversation
if status == 'READY':
    print("🗣️ Testing Multi-turn Conversation")
    print("=" * 50)
    
    conversation = [
        "I want to visit Italy",
        "I prefer mid-range hotels and vegetarian food",
        "Tell me about Rome's attractions",
        "What about Florence?"
    ]
    
    for i, message in enumerate(conversation, 1):
        print(f"\n--- Turn {i} ---")
        test_payload = {"prompt": message}
        invoke_response = agentcore_runtime.invoke(test_payload)
        
        print(f"User: {message}")
        print(f"Agent: {invoke_response['response'][0][:200]}...")  # Truncate for readability

## Step 9: Save Runtime Information

In [ ]:
# Save runtime information for use in subsequent notebooks
import json

if status == 'READY':
    runtime_info = {
        "agent_name": agent_name,
        "agent_arn": launch_result.agent_arn,
        "agent_id": launch_result.agent_id,
        "ecr_uri": launch_result.ecr_uri,
        "region": region,
        "status": status
    }
    
    # Save to file for next notebooks
    with open('environments/runtime_info.json', 'w') as f:
        json.dump(runtime_info, f, indent=2)
    
    print("💾 Runtime information saved to environments/runtime_info.json")
    print("\n📋 Runtime Summary:")
    for key, value in runtime_info.items():
        print(f"  {key}: {value}")
else:
    print("⚠️ Runtime not ready - information not saved")

## Next Steps

✅ **Completed in this notebook:**
- AgentCore Runtime configuration and setup
- Basic travel agent with Strands and Bedrock
- Multi-turn conversation capabilities
- Production deployment to AWS

➡️ **Next: Notebook 03 - Gateway Integration**
- Integrate external APIs (flights, hotels, weather, currency)
- Create OpenAPI 3.0 specifications
- Set up MCP Gateway with OAuth
- Test real API integrations